<h1>Multi-armed Bandits</h1>

# 0. Elements of Reinforcement Learning

+ Policy

+ Reward Signal

+ Value Function

+ Model

# 1. $k$-armed Bandits

The value of action $a$, which is the predicted expectation of rewards, is defined as, 

> $q_*(a) = \mathbb{E}[R_t|A_t = a]$

$A_t$ is the action selected on time step $t$, and $R_t$ is the corresponding reward.